In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
os.chdir("workdir")

from dependencies import (
    load_yaml_from_local
)
from process import (
    process_income,
    process_balance_general,
    process_cash_flow,
    get_financial_score_global,
    process_multiples_price_historic,
    process_compare_multiples_peers
)

In [2]:
config = load_yaml_from_local("config.yaml")
config

INFO:root:Se cargó la configuración en: config.yaml


{'ticker': 'NKE',
 'peers': {'custom': None, 'n_competitors': 5},
 'financial_weights': {'income': {'growth': 0.5, 'peers': 0.5}},
 'multiples_weights': {'pe_ratio': 0.25,
  'ps_ratio': 0.25,
  'pgp_ratio': 0.25,
  'pfcf_ratio': 0.25}}

In [4]:
ticker = config["ticker"]
weights = config["financial_weights"]
peers_cfg = config["peers"]
multiples_weights = config["multiples_weights"]

kpis_cfg = {
    'income': [
        'Revenue',
        'Gross Profit',
        'Operating Income',
        'Net Income'
    ],
    'balance_sheet': [
        'Cash & Equivalents',
        'Cash & Short-Term Investments',
        'Total Current Assets',
        'Total Current Liabilities',
        'Inventory',
        'Total Debt',
        'Total Assets'
    ],
    'cash_flow': [
        'Operating Cash Flow', 
        'Free Cash Flow'
    ],
    "ttm": {
        "income": [
            "Revenue", 
            "Gross Profit", 
            "Net Income", 
            "Shares Outstanding (Basic)"
        ],
        "cash_flow": [
            "Free Cash Flow"
        ]
    }
}

multiples_used = ["pe_ratio", "ps_ratio", "pgp_ratio", "pfcf_ratio"]

response = {
    "financials": {},
    "price_historic": {},
    "price_competitors": {}
}

### Analisis de los estados de resultados de la empresa

In [6]:
results_process_income = process_income(
    ticker, 
    kpis_cfg,
    weights,
    peers_cfg
)
response["financials"]["income"] = results_process_income["income"]
response["peers"] = results_process_income["peers"]

INFO:root:Tipo de resultados: income
INFO:root:KPI's analizar: ['Revenue', 'Gross Profit', 'Operating Income', 'Net Income']

INFO:root:Request successful to url: https://stockanalysis.com/stocks/nke/financials/
INFO:root:earnings date: ['2024-11-30' '2024-05-31' '2023-05-31' '2022-05-31' '2021-05-31'
 '2020-05-31']
INFO:root:Vamos a revisar que la empresa NKE se encuentre creciendo
INFO:root:========================================
INFO:root:Analizando crecimiento de la compañía...
INFO:root:kpi: Revenue analizado. Proporcion de periodos de crecimiento: 0.8
INFO:root:kpi: Gross Profit analizado. Proporcion de periodos de crecimiento: 0.8
INFO:root:kpi: Operating Income analizado. Proporcion de periodos de crecimiento: 0.4
INFO:root:kpi: Net Income analizado. Proporcion de periodos de crecimiento: 0.6
INFO:root:========================================
INFO:root:Score de crecimiento: 6.5
INFO:root:Calculando los margenes de la empresa: NKE
INFO:root:Identificando competidores usando fin

### Balance General

De aqui obtenemos:

1. Razon corriente
2. Razón corriente "acida"
3. Deuda sobre los activos totales
4. Numero de meses de operación con el dinero en caja

In [7]:
response_process_balance = process_balance_general(
    ticker=ticker, 
    kpis=kpis_cfg["balance_sheet"],
    income_stmt_complete=response["financials"]["income"]["income_complete"]
)
response["financials"]["balance"] = response_process_balance["balance"]

INFO:root:Request successful to url: https://stockanalysis.com/stocks/nke/financials/balance-sheet/
INFO:root:Gastos operativos mensuales - operation_expenses_month: 1332
INFO:root:Dinero en caja - cash: 9761.0
INFO:root:Meses de operación con el dinero en caja - months_operation: 7.33
INFO:root:Activos corrientes - current_assets: 24980.0
INFO:root:Pasivos corrientes - current_liabili: 11246.0
INFO:root:current_ratio = current_assets/current_liabili: 2.22
INFO:root:Prueba acida (descontando inventarios de activos corrientes) - quick_ratio: 1.51
INFO:root:Relación deuda vs activos - debt_ratio = total_debt/total_assets: 0.32
INFO:root:========================================
INFO:root:Score final balance general - score_balance: 10.0


### Flujo de caja creciente

In [8]:
response_cash_flow = process_cash_flow(
    ticker=ticker,
    kpis=kpis_cfg["cash_flow"]
)
response["financials"]["cash_flow"] = response_cash_flow["cash_flow"]

INFO:root:Request successful to url: https://stockanalysis.com/stocks/nke/financials/cash-flow-statement/
INFO:root:========================================
INFO:root:Analizando crecimiento de la compañía...
INFO:root:kpi: Operating Cash Flow analizado. Proporcion de periodos de crecimiento: 0.75
INFO:root:kpi: Free Cash Flow analizado. Proporcion de periodos de crecimiento: 0.75
INFO:root:========================================
INFO:root:Score de crecimiento: 7.5


### Score salud financiera global

In [9]:
response["financials"]["score_final"] = get_financial_score_global(
    score_stmt_res=response["financials"]["income"]["score_final"],
    score_balance=response["financials"]["balance"]["score_final"],
    score_cash_flow_growth=response["financials"]["cash_flow"]["score_final"],
    weights=weights
)

INFO:root:score_stmt_res: 5.25
INFO:root:score_balance: 10.0
INFO:root:score_cash_flow_growth: 7.5
INFO:root:Score salud financiera final: 7.58


---

### Análisis del precio historico

In [10]:
response["price_historic"] = process_multiples_price_historic(
    ticker=ticker,
    kpis_cfg=kpis_cfg,
    multiples_used=multiples_used,
    multiples_weights=multiples_weights
)

INFO:root:Obteniendo precio historico para el ticker: NKE
INFO:root:Intervalo de tiempo extraido: 2019-12-23; 2024-12-20
INFO:root:Obteniendo los indicadores de los earnings ttm
INFO:root:Request successful to url: https://stockanalysis.com/stocks/nke/financials/?p=trailing
INFO:root:Estado de resultados historic_income_ttm_interes.shape: (20, 4)
INFO:root:Request successful to url: https://stockanalysis.com/stocks/nke/financials/cash-flow-statement/?p=trailing
INFO:root:Flujo de caja historic_cash_ttm_interes.shape: (19, 1)
INFO:root:Uniendo los estados de resultados y el flujo de caja
INFO:root:historic_ttm_interes.shape: (19, 5)
INFO:root:Replicando los earnings de manera diaria para cruzar con el precio historico
INFO:root:Earnings replicados, hist_ttm_replicated.shape: (1759, 7)
INFO:root:Revisando la moneda de los earnings - earnings_currency: USD
INFO:root:Cruce del precio y los earnings - hist_price_adj.shape: (1258, 4)
INFO:root:Dimension resultante del cruce - hist_price_kpis

# Comparando el precio con la competencia

In [11]:
response["price_competitors"] = process_compare_multiples_peers(
    ticker=ticker,
    hist_multiples_ticker=response["price_historic"]["price"],
    peers=response["peers"],
    kpis_cfg=kpis_cfg,
    multiples_used=multiples_used,
    multiples_weights=multiples_weights
)

INFO:root:========================================
INFO:root:Obteniendo precio historico para el ticker: LULU
INFO:root:Intervalo de tiempo extraido: 2019-12-23; 2024-12-20
INFO:root:Obteniendo los indicadores de los earnings ttm
INFO:root:Request successful to url: https://stockanalysis.com/stocks/lulu/financials/?p=trailing
INFO:root:Estado de resultados historic_income_ttm_interes.shape: (20, 4)
INFO:root:Request successful to url: https://stockanalysis.com/stocks/lulu/financials/cash-flow-statement/?p=trailing
INFO:root:Flujo de caja historic_cash_ttm_interes.shape: (20, 1)
INFO:root:Uniendo los estados de resultados y el flujo de caja
INFO:root:historic_ttm_interes.shape: (20, 5)
INFO:root:Replicando los earnings de manera diaria para cruzar con el precio historico
INFO:root:Earnings replicados, hist_ttm_replicated.shape: (1786, 7)
INFO:root:Revisando la moneda de los earnings - earnings_currency: USD
INFO:root:Cruce del precio y los earnings - hist_price_adj.shape: (1258, 4)
INFO

----

In [12]:
response

{'financials': {'income': {'income_complete':                                     nding Nov 30, 2024 May 31, 2024  \
   0                                 Revenue        48978        51362   
   1                    Revenue Growth (YoY)       -4.97%        0.28%   
   2                         Cost of Revenue        27072        28411   
   3                            Gross Profit        21906        22951   
   4                Selling, General & Admin        15988        16197   
   5                      Operating Expenses        15988        16197   
   6                        Operating Income         5918         6754   
   7                        Interest Expense         -269         -269   
   8            Interest & Investment Income          441          430   
   9           Currency Exchange Gain (Loss)          228          228   
   10  Other Non Operating Income (Expenses)          -49          -27   
   11            EBT Excluding Unusual Items         6269         711